# Реализация и применение алгоритма TransH для построения
# векторного представления графов знаний

Граф мы используем его представление в виде троек h, r, t. тут формула из работы

Основная задача перевести граф в эмбеддирговое пространство

Пару слов о гиперплоскости какие мы будем использовать

плавненько перейти к формуле проектрирования вставить формулу 3



In [3]:
import torch.optim as optim
import numpy as np
import torch
from TransH import TransH

## Модель

### Проекции на гиперплоскость

$ \displaystyle h_{\bot} = h - w_r^{\top}h w_r \\
\displaystyle t_{\bot} = t - w_r^{\top}t w_r $

```
def projected(entity: torch.Tensor, w_r: torch.Tensor):
    w_r = functional.normalize(w_r, dim=-1)
    return entity - torch.sum(entity * w_r, dim=1, keepdim=True) * w_r
```

### Функция оценки

$\displaystyle  f_r(h,t) = \Vert(h - w_r^{\top}h w_r) + d_r - (t - w_r^{\top}t w_r)\Vert_2$

```
 def distance(self, h: torch.Tensor, r: torch.Tensor, t: torch.Tensor):
    head = self.entity_embedding(h)
    w_r = self.w_r_emb(r)
    d_r = self.d_r_emb(r)
    tail = self.entity_embedding(t)

    head_hyper = self.projected(head, w_r)
    tail_hyper = self.projected(tail, w_r)

    distance = head_hyper + d_r - tail_hyper
    score = torch.norm(distance, dim=1)
    return score
```


### Ошибка основанная на ограничении: все эмбеддинги находятся в единичном шаре

$\displaystyle  \forall e \in E,\|\mathbf{e}\|_{2} \leq 1$

```
def scale_loss(embedding: torch.Tensor):
    return torch.sum(torch.relu(torch.norm(embedding, dim=1) - 1))
```


### Ошибка основанная на ограничении: вектор перемещения находится на гиперплоскости.

$ \displaystyle  \forall r \in R:  \frac{\left|\mathbf{w}_{r}^{\top} \mathbf{d}_{r}\right|} {\left\|\mathbf{d}_{r}\right\|_{2} \left\|\mathbf{w}_{r}\right\|_{2}} =  \frac{\left|\mathbf{w}_{r}^{\top} \mathbf{d}_{r}\right|} {\left\|\mathbf{d}_{r}\right\|_{2}}\leq \epsilon$

```
def orthogonal_loss(self, relation_embedding: torch.Tensor, w_embedding: torch.Tensor):
    dot = torch.sum(relation_embedding * w_embedding, dim=1) ** 2
    norm = torch.norm(relation_embedding, dim=1) ** 2
    loss = torch.sum(
        torch.relu(dot / norm - torch.FloatTensor([self.epsilon]) ** 2))
    return loss
```


### Функция потерь с мягкими ограничениями.

$ \displaystyle \mathcal{L} =\sum_{(h, r, t) \in \Delta} \sum_{\left(h^{\prime}, r^{\prime}, t^{\prime}\right) \in \Delta_{(h, r, t)}^{\prime}}\left[f_{r}(\mathbf{h}, \mathbf{t})+\gamma-f_{r^{\prime}}\left(\mathbf{h}^{\prime}, \mathbf{t}^{\prime}\right)\right]_{+}
+C\left\{\sum_{e \in E}\left[\|\mathbf{e}\|_{2}^{2}-1\right]_{+}+\sum_{r \in R}\left[\frac{\left(\mathbf{w}_{r}^{\top} \mathbf{d}_{r}\right)^{2}}{\left\|\mathbf{d}_{r}\right\|_{2}^{2}}-\epsilon^{2}\right]_{+}\right\}$

```
def loss(self, positive_triplets: torch.Tensor, negative_triplets: torch.Tensor):
    h, r, t = torch.chunk(positive_triplets, 3, dim=1)
    h_c, r_c, t_c = torch.chunk(negative_triplets, 3, dim=1)

    positive = self.distance(h, r, t)
    negative = self.distance(h_c, r_c, t_c)

    loss = torch.relu(positive - negative + self.gamma)

    entity_embedding = self.entity_embedding(torch.cat([h, t, h_c, t_c]))
    relation_embedding = self.d_r_emb(torch.cat([r, r_c]))
    w_embedding = self.w_r_emb(torch.cat([r, r_c]))

    orthogonal_loss = self.orthogonal_loss(relation_embedding, w_embedding)

    scale_loss = self.scale_loss(entity_embedding)

    return loss + self.c * (scale_loss / len(entity_embedding) + orthogonal_loss / len(relation_embedding))

```


## Данные
Используем WordNet.
WordNet® - это большая лексическая база данных английского языка. Существительные, глаголы, прилагательные и наречия сгруппированы в наборы когнитивных синонимов (синсетов), каждый из которых выражает отдельное понятие. Синсеты взаимосвязаны с помощью концептуально-семантических и лексических отношений.

#### Пример:
[корги + гипероним = собака]
Гиперо́ним — слово с более широким значением, выражающее общее, родовое понятие. Собака для корги является гиперонимом, корги для собаки является гипонимом
<img src="WordNet18RR/example_screen/wordnextsearch.png" data-canonical-src="WordNet18RR/example_screen/wordnextsearch.png" width="800" />

<img src="WordNet18RR/example_screen/corgi.png"
 data-canonical-src="WordNet18RR/example_screen/corgi.png" width="800" />

[https://wordnet.princeton.edu](https://wordnet.princeton.edu)
[https://pytorch-geometric.readthedocs.io/en/latest/modules/datasets.html#torch_geometric.datasets.WordNet18RR](https://pytorch-geometric.readthedocs.io/en/latest/modules/datasets.html#torch_geometric.datasets.WordNet18RR)

In [4]:
from torch_geometric.datasets import WordNet18RR

WordNet18RR('./WordNet18RR')  # Автоматическая загрузка данных в папку ./WordNet18RR

data = torch.load('./WordNet18RR/processed/data.pt')[0] # обращаемся к первому элементу кортежа, который является torch_geometric.data.data.Data
data

Data(edge_index=[2, 93003], edge_type=[93003], train_mask=[93003], val_mask=[93003], test_mask=[93003], num_nodes=40943)

In [5]:
# ведет себя как именованный кортеж. Можно обращаться к данным внутри как к атрибутам.
entities = torch.arange(0, data.num_nodes)  # создадим id, они занумерованы от 0 до 40943
relations = data.edge_type.unique()  # имеем 11 уникальных связей.

head, tail = data.edge_index[:, data.train_mask] # оставляем только индексы для тренировочных триплетов
relation = data.edge_type[data.train_mask]  # оставляем id для тренировочных триплетов
train_triplets = torch.stack([head, relation, tail]).T # образуется тензор состоящий из трех одномерных тензоров head, relation, tail, по столбцам образовались тройки, при транспонировании получаем тензор где в каждой строке получается тройка.

print('Тренировочных триплетов: ', train_triplets.shape[0])

head_valid, tail_valid = data.edge_index[:, data.val_mask]
relation_valid = data.edge_type[data.val_mask]
valid_triplets = torch.stack([head_valid, relation_valid, tail_valid]).T

print('Валидационных триплетов: ', valid_triplets.shape[0])

head_test, tail_test = data.edge_index[:, data.test_mask]
relation_test = data.edge_type[data.test_mask]
test_triplets = torch.stack([head_test, relation_test, tail_test]).T

print('Тестовых триплетов: ', test_triplets.shape[0])

print(train_triplets.shape[0] + valid_triplets.shape[0] + test_triplets.shape[0])


# Будем вычислять на cuda или cpu 
device = 'cuda' if torch.cuda.is_available() else 'cpu'
train_triplets = train_triplets.to(device)
test_triplets = test_triplets.to(device)
valid_triplets = valid_triplets.to(device)

Тренировочных триплетов:  86835
Валидационных триплетов:  3034
Тестовых триплетов:  3134
93003


## Инициализируем модель

[https://docs.ampligraph.org/en/latest/experiments.html](https://docs.ampligraph.org/en/latest/experiments.html)
настроим параметры примерно, как настраивали наши коллеги

In [4]:
dimension = 256
gamma = 2.0

c = 1.0
epsilon = 1e-5
learning_rate = 0.01

In [5]:
model = TransH(len(entities), len(relations), dimension, gamma, c, epsilon, device=device).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Будем уменьшать lr каждые 50 эпох. После 300 эпох отключим это. 0.1 * 0.5 ** 6 ~ 0.00016
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.5) 

## Подсчет статистик:
1. Среднее количество хвостовых объектов на головной объект ($tph$)

2. Среднее количество головных объектов на хвостовой объект ($hpt$)

In [6]:
prob_tph = torch.zeros(len(relations))
all_relations = data.edge_type
for i in range(len(relations)):
    mask_i = all_relations == i  # получили булев тензор
    head, tail = data.edge_index[:, mask_i]
    amount_head = head.unique().shape[0]
    amount_tail = tail.unique().shape[0]
    tph = amount_tail / amount_head
    hpt = amount_head / amount_tail
    # создаем вектор нормированных статистик tph по формуле **
    prob_tph[i] = tph / (tph + hpt)

prob_tph = prob_tph.to(device)
print(prob_tph)

tensor([0.5647, 0.5000, 0.8075, 0.0677, 0.0249, 0.8548, 0.9840, 0.9985, 0.5000,
        0.0097, 0.5000])


### Создание отрицательных триплетов

In [7]:
def gen_negative_triplets(_positive: torch.Tensor, _entities: torch.Tensor, _prob_tph: torch.Tensor):
    """ Возвращает случайно созданные негативные триплеты """
    
    # тензор _positive лежит в памяти если не сделать копию, то мы будем изменять исходный
    # тензор а не создавать негативный
    _negative = _positive.clone()
    
    p = torch.rand(_negative.shape[0]).to(device)  # генерируем вектор вероятностей 
    pr = _prob_tph[_negative[:, 1]]  # забираем метрику prob tph
    
    mask_replace_head = pr > p  # маска для замены head при условии что pr > p
    
    # заменяем головную сущность
    random_heads = torch.randint(0, _entities.shape[0], (mask_replace_head.sum(), )).to(device)
    _negative[mask_replace_head, 0] = random_heads
    
    # заменяем хвостовую сущность
    random_tails = torch.randint(0, _entities.shape[0], ((mask_replace_head == False).sum(), )).to(device)
    _negative[mask_replace_head == False, 2] = random_tails

    return _negative.to(device)

## Обучение модели

In [8]:
epochs = 4000
batch_size = len(train_triplets)
n_entities = len(entities)

n_batches = len(train_triplets) // batch_size
n_valid_batch = len(valid_triplets) // batch_size + 1
print(n_batches, n_valid_batch)

1 1


In [44]:
%%time
history = {'train history': [], 'validation history': [], 'epoch': []}

least_loss = np.inf  # Будем контролировать обучение. Если эпох без улучшений больше 60 прекращаем тренировку
count_without_update = 0
best_model = model.state_dict()


for epoch in range(epochs):

    epoch_loss = 0
    epoch_valid_loss = 0

    for batch in range(n_batches):

        # на каждой итерации создается новый батч размера batch_size состоящий из случайных триплетов
        positive = train_triplets[torch.randint(0, len(train_triplets), (batch_size, ))] # правильные триплеты
        negative = gen_negative_triplets(positive, entities, prob_tph)  # отрицательные триплеты
        
        optimizer.zero_grad()
        
        loss = model.loss(positive, negative)
        loss.backward() # считает градиент по каждому параметру модели

        optimizer.step()

        epoch_loss += loss.item()
    
    # валидация
    for batch in range(n_valid_batch):
        positive = valid_triplets[torch.randint(0, len(valid_triplets), (batch_size, ))]
        negative = gen_negative_triplets(positive, entities, prob_tph)
        with torch.no_grad():
            epoch_valid_loss += model.loss(positive, negative).item()

    # усредняем ошибку по количеству батчей
    history['train history'].append(epoch_loss / n_batches)
    history['validation history'].append(epoch_valid_loss / n_valid_batch)
    history['epoch'].append(epoch)
    
    if epoch_valid_loss < least_loss:
        least_loss = epoch_valid_loss
        best_model = model.state_dict()
        count_without_update = 0
        
    else: 
        count_without_update += 1
        if count_without_update >= 60:
            model.load_state_dict(best_model)
            break
    
    print(f'epoch: {epoch:4d} | loss: {epoch_loss / n_batches: 0.4f} '
          f'| valid : {epoch_valid_loss / n_valid_batch: 0.4f} '
          f'| lr : {optimizer.param_groups[0]["lr"]}', 
          end='\r')
    
    if epoch < 300:
        scheduler.step()
        
print()

epoch:  342 | loss:  1.4129 | valid :  1.4858 | lr : 0.00015625
Wall time: 7min 28s


In [45]:
# !pip install -U kaleido  # На многих устройствах нет kaleido, чтобы сохранить график

import plotly.express as px
fig = px.line(history, x='epoch', y=['train history', 'validation history'], 
              title='<b>Ошибка на тренировке и валидации на эпоху обучения</b>')

# fig.write_image('./Trained Embeddings/training plots.png', width=1000, height=600, scale=2)
fig.show()

### График полученный на последнем обучении
![last_train](./Trained%20Embeddings/training%20plots.png)

### График обучения лучшей модели
![best_train](./Trained%20Embeddings/best%20training%20plots.png)

### Сохранение обученных весов

In [46]:
# Сохраняем эмбеддинги, ради которых и было обучение 
torch.save(model.entity_embedding.weight.data.cpu(), './Trained Embeddings/entity_embs.pt') 
torch.save(model.w_r_emb.weight.data.cpu(), './Trained Embeddings/w_r_emb (normal_vectors).pt')
torch.save(model.d_r_emb.weight.data.cpu(), './Trained Embeddings/d_r_emb (hyperplane relations).pt')

### Загрузка обученных весов

In [10]:
# Загрузка обученной модели
# torch.load - ....

model.load_trained_embeddings(torch.load('./Trained Embeddings/best entity_embs.pt').to(device),
                              torch.load('./Trained Embeddings/best d_r_emb (hyperplane relations).pt').to(device),
                              torch.load('./Trained Embeddings/best w_r_emb (normal_vectors).pt').to(device))


# Тестирование

In [11]:
%%time
hits = 0

model.eval()  # переводим модель в режим оценки

for i, triplet in enumerate(test_triplets):
    # повторяет triplet построчно в количестве len(entities) = 40943 раз
    head_replaced = torch.tile(triplet, (len(entities), 1))
    head_replaced[:, 0] = entities  # заменяет head на все возможные значения entities

    distance = model.distance(*head_replaced.T)  # вычисляем расстояние для каждого триплета
    # 10 минимальных значений расстояний, для которых мы находим индексы головных сущностей
    heads_top_10 = distance.topk(10, largest=False).indices
    
    # проверяем, что индекс настоящей головной сушности в топ10 для сущностей с минимальным расстоянием
    if triplet[0] in heads_top_10:
        hits += 1

    tail_replaced = torch.tile(triplet, (len(entities), 1))
    tail_replaced[:, 2] = entities

    distance = model.distance(*tail_replaced.T)

    tails_top_10 = distance.topk(10, largest=False).indices
    # проверяем, что индекс настоящей хвостовой сущности в топ10 для сущностей с минимальным расстоянием
    if triplet[2] in tails_top_10:
        hits += 1
        
    print(f'train : {i + 1:4d} | hits : {hits:4d}', end='\r')
    
    
print('')

hit_10 = hits / (2 * len(test_triplets))
print(f'HIT@10 : {hit_10 : 0.3f}\n')

train : 3134 | hits : 2955
HIT@10 :  0.471

CPU times: user 26min 40s, sys: 38min 27s, total: 1h 5min 8s
Wall time: 1h 1min 53s


### Сравнение с другими моделями

![TranH_COMP](./WordNet18RR/example_screen/TranH%20comparison.png)

[https://paperswithcode.com/sota/link-prediction-on-wn18rr](https://paperswithcode.com/sota/link-prediction-on-wn18rr)